### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [4]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
# print(f"NYT API KEY: {nyt_api_key}")

### Access the New York Times API

In [6]:
# Set the base URL
base_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
url = (
    f"{base_url}"
    f"api-key={nyt_api_key}&"
    f"fq={filter_query}&"
    f"sort={sort}&"
    f"fl={field_list}&"
    f"begin_date={begin_date}&"
    f"end_date={end_date}"
)

In [7]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    url = (
        f"{base_url}"
        f"api-key={nyt_api_key}&"
        f"fq={filter_query}&"
        f"sort={sort}&"
        f"fl={field_list}&"
        f"begin_date={begin_date}&"
        f"end_date={end_date}"
        f"page={page}"
    )
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    response = requests.get(url)
    data = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in data["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Checked page {page}")


        # Print the page number that had no results then break from the loop
        if not data["response"]["docs"]:
            print(f"None on page {page}")
            break

    except KeyError:
        break

reviews_df = pd.DataFrame(reviews_list)
print(reviews_df)


Checked page 0


Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19
                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2023/05/04/movies/you-...   
3    https://www.nytimes.com/2023/04/21/movies/a-to...   
4    https://www.nytimes.com/2023/04/20/movies/othe...   
..                                                 ...   
195  https://www.nytimes.com/2023/04/13/movies/one-...   
196  https://www.nytimes.com/2023/04/13/movies/the-...   
197  https://www.nytimes.com/2023/03/30/movies/a-th...   
198  https://www.nytimes.com/2023/02/09/movies/your...   
199  https://www.nytimes.com/2023/02/02/movies/love...   

        

In [25]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
# for i, review in enumerate(reviews_list[:5]):
#     formatted_json = json.dumps(review, indent=4)
#     print(f"Review {i + 1}:\n{formatted_json}\n")

reviews_list = response.json()
print(json.dumps(reviews_list, indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                "source": "The New York Times",
                "headline": {
                    "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "The Attachment Diaries",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                        "ra

In [31]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()

reviews_df = pd.json_normalize(reviews_list["response"]["docs"])

# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)

# print(reviews_df)

reviews_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}]",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
1,https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html,Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'persons', 'value': 'Kapur, Shekhar', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'James, Lily', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Azmi, Shabana', 'rank': 4, 'major': 'N'}, {'name': 'persons', 'value': 'Thompson, Emma', 'rank': 5, 'major': 'N'}]",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Probably a Lot,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
2,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'You Can Live Forever (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Slutsky, Mark', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Watts, Sarah (Film Director)', 'rank': 4, 'major': 'N'}, {'name': 'persons', 'value': 'O'Driscoll, Anwen', 'rank': 5, 'major': 'N'}, {'name': 'persons', 'value': 'Laporte, June (Actor)', 'rank': 6, 'major': 'N'}]",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
3,https://www.nytimes.com/2023/04/21/movies/a-tourists-guide-to-love-review.html,Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'A Tourist's Guide to Love (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Tsuchida, Steven', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Cook, Rachael Leigh', 'rank': 4, 'major': 'N'}]",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None
4,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.",The New Yo

In [61]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
# def extract_title(headline_main):
#     start_marker = '\u2018'
#     end_marker = ' Review\u2019'
#     start = headline_main.find(start_marker)
#     end = headline_main.find(end_marker)
#     if start != -1 and end != -1:
#         return headline.main[start + len(start_marker):end]
#     return None
def extract_title(headline_main):
    start_marker = '‘'  
    end_marker = '’' 
    start = headline_main.find(start_marker)
    end = headline_main.find(end_marker)
    if start != -1 and end != -1:
        return headline_main[start + len(start_marker):end]
    return headline_main

reviews_df['title'] = reviews_df['headline.main'].apply(extract_title)
reviews_df

# print(reviews_df['headline.main'].head(10))

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html,Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.,The New York Times,"subject: Movies;persons: Kapur, Shekhar;persons: James, Lily;persons: Azmi, Shabana;persons: Thompson, Emma;",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Probably a Lot,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"subject: Movies;creative_works: You Can Live Forever (Movie);persons: Slutsky, Mark;persons: Watts, Sarah (Film Director);persons: O'Driscoll, Anwen;persons: Laporte, June (Actor);",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-tourists-guide-to-love-review.html,Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.,The New York Times,"subject: Movies;creative_works: A Tourist's Guide to Love (Movie);persons: Tsuchida, Steven;persons: Cook, Rachael Leigh;",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.",The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;creative_works: Other People's Children (Movie);",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, 'lastname': 'Dargis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,Other People
5,https://www.nytimes.com/2023/04/13/movies/one-true-loves-movie-review.html,"A film adaptation of Taylor Jenkins Reid’s novel has potential for drama, but it stumbles on stock melodrama.",The New York Times,"subject: Movies;persons: Bracey, Luke (1989- );persons: Liu, Simu;persons: Reid, Taylor Jenkins;persons: Soo, Phillipa;persons: Fickman, Andy;creative_works: One T

In [62]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    if isinstance(keyword_list, list):
        extracted_keywords = ""
        for item in keyword_list:
            # Extract 'name' and 'value'
            keyword = f"{item['name']}: {item['value']};" 
            # Append the keyword item to the extracted_keywords list
            extracted_keywords += keyword
        return extracted_keywords
    return keyword_list

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)

reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html,Two childhood friends navigate cultural differences in this pleasantly uncontentious romantic comedy.,The New York Times,"subject: Movies;persons: Kapur, Shekhar;persons: James, Lily;persons: Azmi, Shabana;persons: Thompson, Emma;",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Probably a Lot,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"subject: Movies;creative_works: You Can Live Forever (Movie);persons: Slutsky, Mark;persons: Watts, Sarah (Film Director);persons: O'Driscoll, Anwen;persons: Laporte, June (Actor);",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-tourists-guide-to-love-review.html,Rachael Leigh Cook stars in this bland rom-com as a travel executive exploring Vietnam and getting over a breakup.,The New York Times,"subject: Movies;creative_works: A Tourist's Guide to Love (Movie);persons: Tsuchida, Steven;persons: Cook, Rachael Leigh;",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.",The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;creative_works: Other People's Children (Movie);",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, 'lastname': 'Dargis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,Other People
5,https://www.nytimes.com/2023/04/13/movies/one-true-loves-movie-review.html,"A film adaptation of Taylor Jenkins Reid’s novel has potential for drama, but it stumbles on stock melodrama.",The New York Times,"subject: Movies;persons: Bracey, Luke (1989- );persons: Liu, Simu;persons: Reid, Taylor Jenkins;persons: Soo, Phillipa;persons: Fickman, Andy;creative_works: One T

In [63]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_df['title'].to_list()

print(titles_list)
print(reviews_df[["title"]].head())

['The Attachment Diaries', 'What', 'You Can Live Forever', 'A Tourist', 'Other People', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl']
                    title
0  The Attachment Diaries
1                    What
2    You Can Live Forever
3               A Tourist
4            Other People


### Access The Movie Database API

In [64]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [65]:
# Create an empty list to store the results
movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
counter = 0

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if counter > 0 and counter % 50 == 0:
        time.sleep(12)

    # Add 1 to the request counter
    counter += 1
    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string)
    data = response.json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = data["results"][0]["id"]

        # Make a request for a the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        
        

        # Execute "GET" request with url
        movie_response = requests.get(movie_details_url)
        movie_data = movie_response.json()

        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_data['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in movie_data['spoken_languages']]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_data['production_countries']]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_info = {
            'title': movie_data['title'],
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        movies_list.append(movie_info)
        
        # Print out the title that was found
        print(f"Found {movie_data['title']}")
    
    except (IndexError, KeyError):
        print(f"{title} not found.")

movie_df = pd.DataFrame(movies_list)

movie_df


Found The Attachment Diaries
Found What?
Found You Can Live Forever
Found A Tourist's Guide to Love
Found Other People
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl


,title,genres,spoken_languages,production_countries
0,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,What?,[Comedy],[],[United States of America]
2,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,A Tourist's Guide to Love,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,Other People,"[Comedy, Drama]",[English],[United States of America]
5,One True Loves,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
6,The Lost Weekend: A Love Story,[Documentary],[English],[United States of America]
7,A Thousand and One,"[Drama, Crime]","[English, Spanish, Portuguese]",[United States of America]
8,Your Place or Mine,"[Romance, Comedy]",[English],[United States of America]
9,Love in the Time of Fentanyl,[Documentary],[English],"[Canada, United States of America]"


In [66]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
movie_list = response.json()
print(json.dumps(movie_list, indent=4))

{
    "page": 1,
    "results": [
        {
            "adult": false,
            "backdrop_path": "/lE8xquVrAf5UiUA8pzBxxdp2vPH.jpg",
            "genre_ids": [
                99
            ],
            "id": 976653,
            "original_language": "en",
            "original_title": "Love in the Time of Fentanyl",
            "overview": "An intimate portrait of a community fighting to save lives and keep hope alive in a neighborhood ravaged by the overdose crisis.",
            "popularity": 1.763,
            "poster_path": "/rSAhu53e2WgHZZX2oiXnzxr3Q09.jpg",
            "release_date": "2023-02-03",
            "title": "Love in the Time of Fentanyl",
            "video": false,
            "vote_average": 0.0,
            "vote_count": 0
        }
    ],
    "total_pages": 1,
    "total_results": 1
}


In [67]:
# Convert the results to a DataFrame
movie_df

,title,genres,spoken_languages,production_countries
0,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,What?,[Comedy],[],[United States of America]
2,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,A Tourist's Guide to Love,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,Other People,"[Comedy, Drama]",[English],[United States of America]
5,One True Loves,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
6,The Lost Weekend: A Love Story,[Documentary],[English],[United States of America]
7,A Thousand and One,"[Drama, Crime]","[English, Spanish, Portuguese]",[United States of America]
8,Your Place or Mine,"[Romance, Comedy]",[English],[United States of America]
9,Love in the Time of Fentanyl,[Documentary],[English],"[Canada, United States of America]"


### Merge and Clean the Data for Export

In [69]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, movie_df, on='title', how='inner')

merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"subject: Movies;creative_works: You Can Live Forever (Movie);persons: Slutsky, Mark;persons: Watts, Sarah (Film Director);persons: O'Driscoll, Anwen;persons: Laporte, June (Actor);",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.",The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;creative_works: Other People's Children (Movie);",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, 'lastname': 'Dargis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,Other People,"[Comedy, Drama]",[English],[United States of America]
3,https://www.nytimes.com/2023/04/13/movies/one-true-loves-movie-review.html,"A film adaptation of Taylor Jenkins Reid’s novel has potential for drama, but it stumbles on stock melodrama.",The New York Times,"subject: Movies;persons: Bracey, Luke (1989- );persons: Liu, Simu;persons: Reid, Taylor Jenkins;persons: Soo, Phillipa;persons: Fickman, Andy;creative_works: One True Loves (Movie);",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, 'lastname': 'Yu', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,One True Loves,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
4,https://www.nytimes.com/2023/04/13/movies/the-lost-weekend-a-love-story-review.html,"There’s not much Lennon music heard in this doc about his affair with May Pang, and given how much Pang trashes his wife, Yoko Ono, it’s no surprise it was withheld.",The New York Times,"subject: Documentary Films and Programs;creative_works: The Lost Weekend: A Love Story (Movie);persons: Pang, May;persons: Lennon, John;persons: Ono, Yoko;persons: Brandstein, Eve;persons: Kaufman, Richard S (Filmmaker);persons: Samuels, Stuart (Filmmaker);subject: Content Type: Personal Profile;subject: Pop and Rock Music;",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When John Lennon

In [73]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
chars_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, "", regex=False)

# Display the fixed DataFrame
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"subject: Movies;creative_works: You Can Live Forever (Movie);persons: Slutsky, Mark;persons: Watts, Sarah (Film Director);persons: O'Driscoll, Anwen;persons: Laporte, June (Actor);",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None, 'lastname': 'Vincentelli', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.",The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;creative_works: Other People's Children (Movie);",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, 'lastname': 'Dargis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,Other People,"Comedy, Drama",English,United States of America
3,https://www.nytimes.com/2023/04/13/movies/one-true-loves-movie-review.html,"A film adaptation of Taylor Jenkins Reid’s novel has potential for drama, but it stumbles on stock melodrama.",The New York Times,"subject: Movies;persons: Bracey, Luke (1989- );persons: Liu, Simu;persons: Reid, Taylor Jenkins;persons: Soo, Phillipa;persons: Fickman, Andy;creative_works: One True Loves (Movie);",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, 'lastname': 'Yu', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}]",None,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America"
4,https://www.nytimes.com/2023/04/13/movies/the-lost-weekend-a-love-story-review.html,"There’s not much Lennon music heard in this doc about his affair with May Pang, and given how much Pang trashes his wife, Yoko Ono, it’s no surprise it was withheld.",The New York Times,"subject: Documentary Films and Programs;creative_works: The Lost Weekend: A Love Story (Movie);persons: Pang, May;persons: Lennon, John;persons: Ono, Yoko;persons: Brandstein, Eve;persons: Kaufman, Richard S (Filmmaker);persons: Samuels, Stuart (Filmmaker);subject: Content Type: Personal Profile;subject: Pop and Rock Music;",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When John Lennon Strayed,None,None,The L

In [74]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=['byline.person'])

In [75]:
# Delete duplicate rows and reset index
columns_to_check = [
    'web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
    'headline.main', 'headline.kicker', 'headline.content_kicker',
    'headline.print_headline', 'headline.name', 'headline.seo', 'headline.sub',
    'byline.original', 'byline.organization', 'title', 'genres',
    'spoken_languages', 'production_countries']

merged_df = merged_df.drop_duplicates(subset=columns_to_check)

merged_df = merged_df.reset_index(drop=True)

merged_df.head()



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html,Religion comes between two girls falling in love in the 1990s in this sweet coming-of-age film bathed in grunge hues.,The New York Times,"subject: Movies;creative_works: You Can Live Forever (Movie);persons: Slutsky, Mark;persons: Watts, Sarah (Film Director);persons: O'Driscoll, Anwen;persons: Laporte, June (Actor);",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me Now?,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html,"A radiant Virginie Efira stars as a Parisian teacher who blissfully falls for a man and his 4-year-old daughter, complicating everyone’s lives.",The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;creative_works: Other People's Children (Movie);",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None,Other People,"Comedy, Drama",English,United States of America
3,https://www.nytimes.com/2023/04/13/movies/one-true-loves-movie-review.html,"A film adaptation of Taylor Jenkins Reid’s novel has potential for drama, but it stumbles on stock melodrama.",The New York Times,"subject: Movies;persons: Bracey, Luke (1989- );persons: Liu, Simu;persons: Reid, Taylor Jenkins;persons: Soo, Phillipa;persons: Fickman, Andy;creative_works: One True Loves (Movie);",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America"
4,https://www.nytimes.com/2023/04/13/movies/the-lost-weekend-a-love-story-review.html,"There’s not much Lennon music heard in this doc about his affair with May Pang, and given how much Pang trashes his wife, Yoko Ono, it’s no surprise it was withheld.",The New York Times,"subject: Documentary Films and Programs;creative_works: The Lost Weekend: A Love Story (Movie);persons: Pang, May;persons: Lennon, John;persons: Ono, Yoko;persons: Brandstein, Eve;persons: Kaufman, Richard S (Filmmaker);persons: Samuels, Stuart (Filmmaker);subject: Content Type: Personal Profile;subject: Pop and Rock Music;",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When John Lennon Strayed,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None,The Lost Weekend: A Love Story,Documentary,English,United States of America


In [76]:
# Export data to CSV without the index
merged_df.to_csv("merged_df.csv", index=False)